In [1]:
import pandas as pd
%matplotlib inline

# create df

In [2]:
df = pd.DataFrame(columns=['random_R', 'random_Y', 'random_B', 'expt_R', 'expt_Y', 'expt_B'])

# count row and add into df

In [3]:
def count_each_row(rand_grp, expt_grp, dataf):
    pre_row = [rand_grp.count('r'), rand_grp.count('y'), rand_grp.count('b'), 
               expt_grp.count('r'), expt_grp.count('y'), expt_grp.count('b')]
    dataf = dataf.append(pd.DataFrame([pre_row], 
                                      columns=['random_R', 'random_Y', 'random_B', 'expt_R', 'expt_Y', 'expt_B']), 
                         ignore_index=True)
    return dataf

# add true pop to df

In [4]:
# make two populations n=18 each: R/Y/B: random = 6/6/6; experimental = 15/0/3
random_group = ['r']*6 + ['y']*6 + ['b']*6
expt_group = ['r']*15 + ['y']*0 + ['b']*3

# add to df
df = count_each_row(random_group, expt_group, df)

# get 500 fake populations

In [6]:
from random import shuffle

In [7]:
# merge populations
merge_grp = random_group + expt_group

In [8]:
for i in range(500):
    shuffle(merge_grp)
    df = count_each_row(merge_grp[:18], merge_grp[18:], df)

# get fisher p values

In [10]:
from scipy import stats

In [11]:
# fisher 2x3:
import matlab.engine
eng = matlab.engine.start_matlab()
try:
    eng.addpath(r'/Users/ZhiqiYip/Documents/MATLAB/Code')
except AttributeError:
    eng.addpath(r'/Users/hink_pink/Documents/MATLAB/Code')

In [12]:
# make MATLAB array
fisher_p_values = []

for j in range(501):
    array_for_fisher = matlab.double([df.iloc[j].tolist()[:3],
                                      df.iloc[j].tolist()[3:]
                                     ])
    
    # get fisher p-values
    fisher_p_value = eng.myfisher23(array_for_fisher)
    fisher_p_values.append(fisher_p_value)

In [13]:
df['fisher_p_values'] = fisher_p_values

In [17]:
df_sorted = df.sort_values(by='fisher_p_values')
df_sorted['rank'] = range(1,502)
df = df_sorted.sort_index()

In [31]:
df

,random_R,random_Y,random_B,expt_R,expt_Y,expt_B,fisher_p_values,rank
0,6,6,6,15,0,3,0.002650,2
1,10,2,6,11,4,3,0.376975,204
2,7,5,6,14,1,3,0.045092,20
3,8,6,4,13,0,5,0.032580,13
4,10,2,6,11,4,3,0.376975,205
5,13,2,3,8,4,6,0.299751,163
6,10,3,5,11,3,4,1.000000,419
7,9,2,7,12,4,2,0.183349,95
8,11,2,5,10,4,4,0.804114,377
9,11,3,4,10,3,5,1.000000,416


In [35]:
df.to_csv('fig1Ci_data_for_max.csv')